# Replication with Actual Data

This notebook replicates the paper's econometric analysis using the **actual** panel dataset built from FIFA rankings, club rankings, and World Development Indicators.

**Data Sources**:
- FIFA World Rankings (1993-2010)
- FIFA Club Rankings (2000-2010)
- World Bank WDI (1993-2010)

**Output**: Regression results for equations (1), (2), and (3) saved to CSV files.

**Project docs**: [README.md](../README.md), [Data Dictionary](../docs/data-dictionary.md), [Data Pipeline](../docs/data-pipeline.md)

In [1]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from IPython.display import display
from linearmodels.panel import PanelOLS
from linearmodels.iv import IV2SLS

print(f"Python: {sys.version}")
print(f"Pandas: {pd.__version__}")
print(f"Working directory: {Path.cwd()}")

Python: 3.14.2 (main, Dec  5 2025, 16:49:16) [Clang 17.0.0 (clang-1700.6.3.2)]
Pandas: 3.0.0
Working directory: /Users/dkritz/git/fifa-paper/notebooks


## Load Actual Panel Data

Load the panel dataset built from FIFA points, club rankings, and WDI data (1993-2010).

In [2]:
# Load the panel dataset
data_path = Path("../data/analysis/panel.csv")

if not data_path.exists():
    raise FileNotFoundError(f"Panel data not found at {data_path}. Run: python3 scripts/build_panel.py")

df = pd.read_csv(data_path)
print(f"Loaded {len(df)} observations")
print(f"Countries: {df['country'].nunique()}")
print(f"Years: {df['year'].min()}-{df['year'].max()}")
print(f"Variables: {list(df.columns)}")
print("\nConfederations:")
print(df['confed'].value_counts())

Loaded 3276 observations
Countries: 182
Years: 1993-2010
Variables: ['country', 'year', 'confed', 'fifa_points', 'gdp_pc', 'gdp_pc_sq', 'pop', 'pop_sq', 'trade', 'infl', 'oil', 'leb', 'club', 'urbpop', 'urbpop_sq']

Confederations:
confed
CAF         918
UEFA        810
AFC         720
CONCACAF    522
CONMEBOL    180
OFC         126
Name: count, dtype: int64


### Data Snapshot

In [3]:
# Display first few rows and summary stats
display(df.head(10))
print(f"\nDataset shape: {df.shape}")

# Check for missing values
missing = df.isnull().sum()
print("\nMissing values:")
print(missing[missing > 0])

,country,year,confed,fifa_points,gdp_pc,gdp_pc_sq,pop,pop_sq,trade,infl,oil,leb,club,urbpop,urbpop_sq
0,Albania,1993,UEFA,15.0,380.808911,1.450154e+05,3224901.0,1.039999e+13,77.715069,85.004751,4.245053,71.487000,NaN,37.90,1436.4100
1,Albania,1994,UEFA,15.0,624.535068,3.900441e+05,3179442.0,1.010885e+13,50.302012,22.565053,2.309100,71.617732,NaN,38.40,1474.5600
2,Albania,1995,UEFA,15.0,771.862535,5.957718e+05,3141102.0,9.866522e+12,46.987422,7.793219,2.034820,71.870293,NaN,38.90,1513.2100
3,Albania,1996,UEFA,21.0,968.071952,9.371633e+05,3112597.0,9.688260e+12,47.547304,12.725478,1.912895,72.241073,NaN,39.46,1557.0916
4,Albania,1997,UEFA,21.0,710.314788,5.045471e+05,3091902.0,9.559858e+12,47.210926,33.180274,1.783663,72.696073,NaN,40.02,1601.6004
5,Albania,1998,UEFA,24.0,885.908631,7.848341e+05,3079037.0,9.480469e+12,45.134910,20.642859,0.815573,73.197805,NaN,40.58,1646.7364
6,Albania,1999,UEFA,29.0,1117.705767,1.249266e+06,3072725.0,9.441639e+12,49.315913,0.389438,0.873985,73.721829,NaN,41.14,1692.4996
7,Albania,2000,UEFA,428.0,1200.137437,1.440330e+06,3071856.0,9.436299e+12,56.625339,0.050018,1.436102,74.238732,0.0,41.70,1738.8900
8,Albania,2001,UEFA,480.0,1329.385031,1.767265e+06,3077378.0,9.470255e+12,59.230181,3.107588,1.029290,74.723659,0.0,42.32,1790.9824
9,Albania,2002,UEFA,448.0,1440.030143,2.073687e+06,3089778.0,9.546728e+12,67.135909,7.770526,1.219249,75.161220,0.0,42.94,1843.8436



Dataset shape: (3276, 15)

Missing values:
fifa_points      96
gdp_pc          111
gdp_pc_sq       111
pop               2
pop_sq            2
trade           232
infl            401
oil             914
leb             109
club           1274
dtype: int64


In [4]:
# Missingness summary (rates)
missing_counts = df.isnull().sum()
missing_rates = (missing_counts / len(df)).sort_values(ascending=False)
summary = pd.DataFrame({"missing_count": missing_counts, "missing_rate": missing_rates})
display(summary[summary['missing_count'] > 0])

,missing_count,missing_rate
club,1274,0.388889
fifa_points,96,0.029304
gdp_pc,111,0.033883
gdp_pc_sq,111,0.033883
infl,401,0.122405
leb,109,0.033272
oil,914,0.278999
pop,2,0.000611
pop_sq,2,0.000611
trade,232,0.070818


## Load Model Runner Helpers

Import the regression functions from the replication script.

In [ ]:
# Add repo root to path
repo_root = Path.cwd().parent
sys.path.append(str(repo_root))

try:
    from scripts.replicate_stata import fit_fe_ols as _fit_fe_ols, fit_fe_iv as _fit_fe_iv
    print("Successfully imported fit_fe_ols and fit_fe_iv from scripts.replicate_stata")
except Exception as e:
    print(f"Import failed: {e}")
    print("Using notebook implementations...")

def demean_by_entity(df, cols, entity):
    out = df.copy()
    for col in cols:
        ent_mean = out.groupby(entity)[col].transform("mean")
        out[col] = out[col] - ent_mean
    return out

def fit_fe_ols(panel, dep, exog):
    y = panel[dep]
    X = panel[exog]
    model = PanelOLS(y, X, entity_effects=True, time_effects=False)
    return model.fit(cov_type="clustered", cluster_entity=True, debiased=True)

def fit_fe_iv(df, dep, exog, endog, instr, entity, time):
    cols = [dep] + exog + [endog] + instr + [entity, time]
    work = df[cols].dropna().copy()
    work = demean_by_entity(work, [dep] + exog + [endog] + instr, entity)
    y = work[dep]
    X = work[exog]
    endog_v = work[endog]
    Z = work[instr]
    model = IV2SLS(y, X, endog_v, Z)
    return model.fit(cov_type="clustered", clusters=work[entity], debiased=True)

## Equation (1): Baseline Model

**Specification**: FIFA points ~ GDP per capita + GDP per capita² + Population + Population²

**Estimation**: Fixed-effects OLS with country effects and clustered standard errors

In [ ]:
# Define variables
dep = "fifa_points"
base = ["gdp_pc", "gdp_pc_sq", "pop", "pop_sq"]

# Prepare panel
panel = df.set_index(["country", "year"])

# Run Equation (1)
print("="*80)
print("EQUATION (1): Baseline Model")
print("="*80)
res1 = fit_fe_ols(panel, dep, base)
print(res1.summary)
display(res1.summary)

## Equation (2): Model with Macro/Resource Controls

**Specification**: Equation (1) + Trade + Inflation + Oil rents + Life expectancy

**Estimation**: Fixed-effects OLS with country effects and clustered SEs

In [ ]:
# Define additional controls
macro = ["trade", "infl", "oil", "leb"]

# Run Equation (2)
print("="*80)
print("EQUATION (2): Model with Controls")
print("="*80)
res2 = fit_fe_ols(panel, dep, base + macro)
print(res2.summary)
display(res2.summary)

## Equation (3): IV/2SLS with Club Strength

**Specification**: Equation (2) + Club strength (instrumented by urban population + urban population²)

**Estimation**: Fixed-effects IV/2SLS with country effects and clustered SEs

**Instruments**: urbpop, urbpop_sq (for endogenous "club" variable)

In [ ]:
# Define IV variables
club = "club"
instr = ["urbpop", "urbpop_sq"]

# Check club data availability
club_available = df[club].notna().sum()
club_total = len(df)
print(f"\nClub data availability: {club_available}/{club_total} ({100*club_available/club_total:.1f}%)")

# Run Equation (3) if we have club data
if club_available > 0:
    print("="*80)
    print("EQUATION (3): IV/2SLS with Club Strength")
    print("="*80)
    try:
        res3 = fit_fe_iv(df, dep, base + macro, club, instr, "country", "year")
        print(res3.summary)
        display(res3.summary)
    except Exception as e:
        print(f"Error running IV regression: {e}")
        print("\nThis may occur if:")
        print("- All instruments are absorbed by fixed effects")
        print("- No variation in instruments after demeaning")
else:
    print("No club data available - skipping Equation (3)")

## Confederation-Level Analysis

Run regressions separately for each football confederation.

In [ ]:
# Run by confederation
if "confed" in df.columns:
    confederations = sorted(df["confed"].dropna().unique())
    
    for confed in confederations:
        subset = df[df["confed"] == confed]
        if subset.empty:
            continue
        
        print(f"\n{'='*60}")
        print(f"CONFEDERATION: {confed}")
        print(f"Observations: {len(subset)}")
        print(f"Countries: {subset['country'].nunique()}")
        print(f"{'='*60}")
        
        panel_sub = subset.set_index(["country", "year"])
        
        # Equation (1)
        try:
            res1c = fit_fe_ols(panel_sub, dep, base)
            print(f"\nEquation (1) - {confed}")
            print(res1c.summary.tables[1])
        except Exception as e:
            print(f"\nEquation (1) failed for {confed}: {e}")
        
        # Equation (2)
        try:
            res2c = fit_fe_ols(panel_sub, dep, base + macro)
            print(f"\nEquation (2) - {confed}")
            print(res2c.summary.tables[1])
        except Exception as e:
            print(f"\nEquation (2) failed for {confed}: {e}")
        
        # Equation (3) - typically only for UEFA
        if confed.upper() == "UEFA" and club_available > 0:
            try:
                res3c = fit_fe_iv(subset, dep, base + macro, club, instr, "country", "year")
                print(f"\nEquation (3) - {confed}")
                print(res3c.summary.tables[1])
            except Exception as e:
                print(f"\nEquation (3) failed for {confed}: {e}")

## Export Results

Save regression results to CSV files for further analysis.

In [ ]:
# Create results directory
results_dir = Path("results")
results_dir.mkdir(exist_ok=True)

def save_summary(res, path):
    """Save regression summary to CSV."""
    table = res.summary.tables[1].as_csv()
    with open(path, "w") as f:
        f.write(table)
    print(f"Saved: {path}")

# Save full sample results
try:
    save_summary(res1, results_dir / "eq1_full.csv")
    save_summary(res2, results_dir / "eq2_full.csv")
    if 'res3' in dir():
        save_summary(res3, results_dir / "eq3_full.csv")
except Exception as e:
    print(f"Error saving results: {e}")

print(f"\nResults saved to {results_dir}/")
print(f"Files: {list(results_dir.glob('*.csv'))}")

## Summary and Comparison

### Key Findings from Actual Data

Compare these results to the synthetic data notebook and eventually to the original paper.

In [ ]:
# Summary statistics
print("DATASET SUMMARY")
print("="*60)
print(f"Total observations: {len(df)}")
print(f"Countries: {df['country'].nunique()}")
print(f"Year range: {df['year'].min()}-{df['year'].max()}")
print(f"Balanced panel: {len(df) / (df['country'].nunique() * df['year'].nunique()):.1%} complete")

print("\nCONFEDERATION DISTRIBUTION:")
print(df['confed'].value_counts())

print("\nDEPENDENT VARIABLE (fifa_points):")
print(df['fifa_points'].describe())

print("\nKEY INDEPENDENT VARIABLES:")
for var in ['gdp_pc', 'pop', 'club', 'urbpop']:
    if var in df.columns:
        missing = df[var].isnull().sum()
        print(f"  {var}: {missing}/{len(df)} missing ({100*missing/len(df):.1f}%)")

## Next Steps

### Validation Tasks
- [ ] Compare coefficients to original paper
- [ ] Verify sign and significance match expectations
- [ ] Check standard error magnitudes
- [ ] Validate R-squared values

### Data Quality Checks
- [ ] Investigate countries with missing club data
- [ ] Verify country name mappings
- [ ] Check for outliers in FIFA points
- [ ] Validate WDI data ranges

### Extensions
- [ ] Run placebo tests
- [ ] Test robustness to different time periods
- [ ] Add additional controls
- [ ] Export formatted tables

### For Comparison to Paper
1. Locate original Stata output files (if available)
2. Create coefficient comparison table
3. Document any discrepancies
4. Investigate sources of difference (Stata 8/9 defaults, data revisions, etc.)